In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import h3
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from src.settings import *
from shapely.geometry import Polygon
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from src.tools.h3_utils import get_hexagons_for_edges, get_resolution_buffered_suffix
from src.tools.osmnx_utils import generate_data_for_place, get_place_dir_name
from src.tools.feature_extraction import generate_features_for_edges
from pathlib import Path

In [33]:

place_dir = Path(GENERATED_DATA_DIR).joinpath(get_place_dir_name("Wrocław,Poland"))
network_type = "drive"
resolution = 9
buffered = True

shp_dir = join(place_dir, f"shp_{network_type}")
edges_path = join(shp_dir, "edges.shp")
hexagons_path = join(place_dir, f"hex_{get_resolution_buffered_suffix(resolution, buffered)}.geojson")

hexagons: gpd.GeoDataFrame = gpd.read_file(hexagons_path, driver="GeoJSON")  # type: ignore
edges: gpd.GeoDataFrame = gpd.read_file(edges_path)  # type: ignore




In [35]:
edges

,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,from,to,ref,bridge,access,junction,width,tunnel,geometry
0,95584835,297779709,0,"[24707867, 656793686]",1,2,Bardzka,tertiary,50,178.450,95584835,297779709,None,None,None,None,None,None,"LINESTRING (17.04951 51.08311, 17.04947 51.083..."
1,95584835,241778108,0,"[656794402, 28458110]",1,2,Aleja Armii Krajowej,primary,50,28.882,95584835,241778108,94,None,None,None,None,None,"LINESTRING (17.04951 51.08311, 17.04933 51.083..."
2,95584835,95584852,0,"[301726798, 189513998, 301726791]",1,"['2', '3', '4']",Aleja Armii Krajowej,primary,50,284.438,95584852,95584835,94,None,None,None,None,None,"LINESTRING (17.05357 51.08301, 17.05335 51.082..."
3,95584835,3019848617,0,28458112,1,3,Bardzka,secondary,50,25.202,3019848617,95584835,395,None,None,None,None,None,"LINESTRING (17.04960 51.08289, 17.04957 51.082..."
4,95584841,164895026,0,27665976,0,None,Biskupa Bernarda Bogedaina,unclassified,None,14.714,164895026,95584841,None,None,None,None,None,None,"LINESTRING (17.06450 51.08460, 17.06437 51.08470)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9678,8001157238,8221846043,0,"[884068441, 769929444, 727134918, 508232269, 7...",1,3,Nabycińska,tertiary,None,186.065,8001157238,8221846043,None,None,None,None,None,None,"LINESTRING (17.01744 51.11012, 17.01741 51.110..."
9679,8001157238,8001157243,0,858291159,1,1,None,tertiary_link,None,32.034,8001157243,8001157238,None,None,None,None,None,None,"LINESTRING (17.01783 51.11021, 17.01775 51.110..."
9680,8150476313,8150476314,0,875668659,0,None,None,living_street,20,103.432,8150476314,8150476313,None,None,None,None,None,None,"LINESTRING (17.04155 51.05842, 17.04163 51.058..."
9681,8570024805,8570024807,0,"[923192968, 923192971]",0,None,None,living_street,20,162.860,8570024807,8570024805,None,None,None,None,None,None,"LINESTRING (17.13918 51.16063, 17.13949 51.160..."


In [59]:

gpd.sjoin(edges, hexagons[["h3_id", "geometry"]], op="intersects", how="inner") \
    .drop(columns="index_right") \
    .reset_index() \
    .sort_values(by="index", ignore_index=True) \
    .rename(columns={"index": "id"})
# edges_min["h3_id"] = get_hexagons_for_edges(edges_min, hexagons)
# edges_min = edges_min.explode("h3_id").reset_index()  # type: ignore
# edges_min.rename(columns={"index": "id"}, inplace=True)
# edges_min


,id,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,...,from,to,ref,bridge,access,junction,width,tunnel,geometry,h3_id
0,0,95584835,297779709,0,"[24707867, 656793686]",1,2,Bardzka,tertiary,50,...,95584835,297779709,None,None,None,None,None,None,"LINESTRING (17.04951 51.08311, 17.04947 51.083...",891e2040b03ffff
1,1,95584835,241778108,0,"[656794402, 28458110]",1,2,Aleja Armii Krajowej,primary,50,...,95584835,241778108,94,None,None,None,None,None,"LINESTRING (17.04951 51.08311, 17.04933 51.083...",891e2040b03ffff
2,2,95584835,95584852,0,"[301726798, 189513998, 301726791]",1,"['2', '3', '4']",Aleja Armii Krajowej,primary,50,...,95584852,95584835,94,None,None,None,None,None,"LINESTRING (17.05357 51.08301, 17.05335 51.082...",891e2040b07ffff
3,2,95584835,95584852,0,"[301726798, 189513998, 301726791]",1,"['2', '3', '4']",Aleja Armii Krajowej,primary,50,...,95584852,95584835,94,None,None,None,None,None,"LINESTRING (17.05357 51.08301, 17.05335 51.082...",891e2040b03ffff
4,3,95584835,3019848617,0,28458112,1,3,Bardzka,secondary,50,...,3019848617,95584835,395,None,None,None,None,None,"LINESTRING (17.04960 51.08289, 17.04957 51.082...",891e2040b03ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14667,9678,8001157238,8221846043,0,"[884068441, 769929444, 727134918, 508232269, 7...",1,3,Nabycińska,tertiary,None,...,8001157238,8221846043,None,None,None,None,None,None,"LINESTRING (17.01744 51.11012, 17.01741 51.110...",891e2040c67ffff
14668,9679,8001157238,8001157243,0,858291159,1,1,None,tertiary_link,None,...,8001157243,8001157238,None,None,None,None,None,None,"LINESTRING (17.01783 51.11021, 17.01775 51.110...",891e2040c67ffff
14669,9680,8150476313,8150476314,0,875668659,0,None,None,living_street,20,...,8150476314,8150476313,None,None,None,None,None,None,"LINESTRING (17.04155 51.05842, 17.04163 51.058...",891e204e093ffff
14670,9681,8570024805,8570024807,0,"[923192968, 923192971]",0,None,None,living_street,20,...,8570024807,8570024805,None,None,None,None,None,None,"LINESTRING (17.13918 51.16063, 17.13949 51.160...",891e20462c3ffff
